# Data Normalization Script

Applies a series of actions to improve the data usability of the extracted information from leboncoin. 

The script follows the order:

1. Fills missing values
2. Splits the status column into two columns:
    - "Actif depuis le 12/11/2021" -> {"status_type": "Actif", "active_since": 12/11/2021}
3. Extracts the amount of cars from the "no_cars" column.
    - "Annonces (11)" -> 11
4. Transforms the "working_hours" column into two columns:
    - "Du mardi au samedi de 09h00 à 12h00 et de 14h00 à 18h00":
    - {"weekday_range": "tue-sat", "hours": "09:00 - 18:00"}

In [1]:
import pandas as pd

companies_df = pd.read_csv('../../extracted_data/leboncoin/companies.csv', index_col=[0])
companies_df.head()

,name,street,zip,city,siren,status,description,working_hours,no_cars
0,007AUTOMOTIVE,11 rue des deux boulevards,94100.0,Saint-Maur-des-Fossés,903155968,Actif depuis le 12/11/2021,"007 Automotive, c’est un professionnel qui veu...",Lundi au Samedi 09h00-18h00\nPossibilité de pr...,Annonces (11)
1,03 ABS AUTO,14 place de blanzat,3100.0,Montluçon,811944909,Actif depuis le 20/09/2015,Le garage 03 ABS AUTO est spécialisé dans la v...,Disponible sur RENDEZ-VOUS DU LUNDI AU SAMEDI ...,Annonces (81)
2,06MOTORS,149 IMPASSE DES SERRES,6670.0,Saint-Martin-du-Var,930640362,Actif depuis le 02/08/2024,🏍️06Motors est une société de passionnés dédié...,sur rendez-vous,Annonces (10)
3,1000 AUTOS,10 rue du Dronckaert,59223.0,Roncq,813886603,Actif depuis le 24/05/2024,Bienvenue chez 1000 AUTOS \n1000 AUTOS est spé...,Lundi au samedi de 09H00 à 12H00 et de 14H00 à...,Annonces (49)
4,100% MOTO,1&3 rue pierre loti,42100.0,ST ETIENNE,452069610,Actif depuis le 02/11/2010,DUCATI SAINT-ETIENNE\n\nDÉCOUVREZ NOS OCCASION...,du mardi au vendredi: 09h00-12h00 14h00-19h00\...,Annonces (28)


In [2]:
# fill missing values and fix relevant dtypes
companies_df['working_hours'] = companies_df['working_hours'].fillna('')
companies_df['working_hours'] = companies_df['working_hours'].astype('string')
companies_df['zip'] = companies_df['zip'].astype("Int64")

In [3]:
# split the status column into two and remove status column
companies_df['status_type'] = companies_df['status'].str.split().str[0]
companies_df['active_since'] = companies_df['status'].str.extract(r'(\d{2}/\d{2}/\d{4})')

companies_df = companies_df.drop(columns=['status'])
companies_df.head()

,name,street,zip,city,siren,description,working_hours,no_cars,status_type,active_since
0,007AUTOMOTIVE,11 rue des deux boulevards,94100,Saint-Maur-des-Fossés,903155968,"007 Automotive, c’est un professionnel qui veu...",Lundi au Samedi 09h00-18h00 Possibilité de pre...,Annonces (11),Actif,12/11/2021
1,03 ABS AUTO,14 place de blanzat,3100,Montluçon,811944909,Le garage 03 ABS AUTO est spécialisé dans la v...,Disponible sur RENDEZ-VOUS DU LUNDI AU SAMEDI ...,Annonces (81),Actif,20/09/2015
2,06MOTORS,149 IMPASSE DES SERRES,6670,Saint-Martin-du-Var,930640362,🏍️06Motors est une société de passionnés dédié...,sur rendez-vous,Annonces (10),Actif,02/08/2024
3,1000 AUTOS,10 rue du Dronckaert,59223,Roncq,813886603,Bienvenue chez 1000 AUTOS \n1000 AUTOS est spé...,Lundi au samedi de 09H00 à 12H00 et de 14H00 à...,Annonces (49),Actif,24/05/2024
4,100% MOTO,1&3 rue pierre loti,42100,ST ETIENNE,452069610,DUCATI SAINT-ETIENNE\n\nDÉCOUVREZ NOS OCCASION...,du mardi au vendredi: 09h00-12h00 14h00-19h00 ...,Annonces (28),Actif,02/11/2010


In [4]:
# extract the number of cars from the string
companies_df['number_cars'] = companies_df['no_cars'].str.extract(r'\((\d+)\)').fillna(0).astype(int)
companies_df = companies_df.drop(columns=['no_cars'])
companies_df.head()

,name,street,zip,city,siren,description,working_hours,status_type,active_since,number_cars
0,007AUTOMOTIVE,11 rue des deux boulevards,94100,Saint-Maur-des-Fossés,903155968,"007 Automotive, c’est un professionnel qui veu...",Lundi au Samedi 09h00-18h00 Possibilité de pre...,Actif,12/11/2021,11
1,03 ABS AUTO,14 place de blanzat,3100,Montluçon,811944909,Le garage 03 ABS AUTO est spécialisé dans la v...,Disponible sur RENDEZ-VOUS DU LUNDI AU SAMEDI ...,Actif,20/09/2015,81
2,06MOTORS,149 IMPASSE DES SERRES,6670,Saint-Martin-du-Var,930640362,🏍️06Motors est une société de passionnés dédié...,sur rendez-vous,Actif,02/08/2024,10
3,1000 AUTOS,10 rue du Dronckaert,59223,Roncq,813886603,Bienvenue chez 1000 AUTOS \n1000 AUTOS est spé...,Lundi au samedi de 09H00 à 12H00 et de 14H00 à...,Actif,24/05/2024,49
4,100% MOTO,1&3 rue pierre loti,42100,ST ETIENNE,452069610,DUCATI SAINT-ETIENNE\n\nDÉCOUVREZ NOS OCCASION...,du mardi au vendredi: 09h00-12h00 14h00-19h00 ...,Actif,02/11/2010,28


In [5]:
import re
from typing import Tuple
from datetime import datetime

def parse_working_hours(text: str) -> Tuple[str, str]:
    '''
    Parse opening hours text to extract weekday ranges and hours.
    Returns the weekday range and the earliest opening to latest closing time.
    '''
    # Normalize text
    text = text.lower()
    text = re.sub(r'[àâ]', 'a', text)
    text = text.replace('h', ':')
    
    # Common patterns for days, including abbreviations
    days_pattern = {
        'lundi': 'mon',
        'mardi': 'tue',
        'mercredi': 'wed',
        'jeudi': 'thu',
        'vendredi': 'fri',
        'samedi': 'sat',
        'dimanche': 'sun',
        'lun': 'mon',
        'mar': 'tue',
        'mer': 'wed',
        'jeu': 'thu',
        'ven': 'fri',
        'sam': 'sat',
        'dim': 'sun'
    }

    def is_valid_time(hours: int, minutes: int) -> bool:
        '''Check if hours and minutes are valid.'''
        return 0 <= hours <= 23 and 0 <= minutes <= 59
    
    def normalize_time(time_str: str) -> str:
        '''Normalize time format to HH:MM.'''
        try:
            # Remove any spaces
            time_str = time_str.strip().replace(' ', '')
            
            # Handle times ending with ':'
            if time_str.endswith(':'):
                time_str += '00'
                
            # Handle case where we have 'h'
            if 'h' in time_str:
                time_str = time_str.replace('h', ':')

            # Handle case where we have only hours
            if ':' not in time_str:
                hours = int(time_str)
                if not is_valid_time(hours, 0):
                    return None
                return f'{hours:02d}:00'
            
            # Handle HH:MM format
            hours, minutes = map(int, time_str.split(':'))
            if not is_valid_time(hours, minutes):
                return None
            return f'{hours:02d}:{minutes:02d}'
            
        except (ValueError, TypeError):
            return None
    
    def extract_day_range(text: str) -> str:
        '''Extract the range of working days.'''
        # Look for weekday range pattern
        weekday_range_pattern = r'(?:du\s+)?(\w+)\s+(?:au|->|\s*-\s*)\s+(\w+)'
        weekday_matches = re.findall(weekday_range_pattern, text)
        
        if weekday_matches:
            for start_day, end_day in weekday_matches:
                if start_day in days_pattern and end_day in days_pattern:
                    return f'{days_pattern[start_day]}-{days_pattern[end_day]}'
        
        # Look for individual days (including abbreviations)
        days_found = []
        for day in days_pattern.keys():
            if f' {day}' in f' {text}':  # Add spaces to avoid partial matches
                days_found.append(days_pattern[day])
        
        # Check for "rdv" or "rendez-vous"
        if 'rdv' in text or 'rendez-vous' in text:
            if not days_found:
                return 'rdv'
        
        if days_found:
            return ','.join(sorted(set(days_found)))
        return 'unknown'
    
    def extract_hours(text: str) -> str:
        '''Extract earliest opening and latest closing hours.'''
        # Single pattern with more flexible matching
        time_pattern = r'(\d{1,2}[h:]\d{0,2})\s*(?:a|à|-|/|et|de|\s)\s*(\d{1,2}[h:]\d{0,2})'
        
        # Find all time ranges in the text
        times = re.findall(time_pattern, text)
        if not times:
            return 'unknown'
        
        # Get all valid times and normalize them
        all_times = []
        for start, end in times:
            start_time = normalize_time(start)
            end_time = normalize_time(end)
            if start_time and end_time:  # Only add if both times are valid
                all_times.extend([start_time, end_time])
        
        if not all_times:  # If no valid times were found
            return 'unknown'
        
        # Find earliest and latest times
        earliest = min(all_times, key=lambda x: datetime.strptime(x, '%H:%M'))
        latest = max(all_times, key=lambda x: datetime.strptime(x, '%H:%M'))
        
        return f'{earliest} - {latest}'
    
    weekday_range = extract_day_range(text)
    hours = extract_hours(text)
    
    return weekday_range, hours

In [6]:
# applies the parse_working_hours function to transform working_hours into two descriptive columns
companies_df['weekday_range'], companies_df['hours'] = zip(*companies_df['working_hours'].apply(parse_working_hours))
companies_df.head()

,name,street,zip,city,siren,description,working_hours,status_type,active_since,number_cars,weekday_range,hours
0,007AUTOMOTIVE,11 rue des deux boulevards,94100,Saint-Maur-des-Fossés,903155968,"007 Automotive, c’est un professionnel qui veu...",Lundi au Samedi 09h00-18h00 Possibilité de pre...,Actif,12/11/2021,11,mon-sat,09:00 - 18:00
1,03 ABS AUTO,14 place de blanzat,3100,Montluçon,811944909,Le garage 03 ABS AUTO est spécialisé dans la v...,Disponible sur RENDEZ-VOUS DU LUNDI AU SAMEDI ...,Actif,20/09/2015,81,mon-sat,08:30 - 18:30
2,06MOTORS,149 IMPASSE DES SERRES,6670,Saint-Martin-du-Var,930640362,🏍️06Motors est une société de passionnés dédié...,sur rendez-vous,Actif,02/08/2024,10,rdv,unknown
3,1000 AUTOS,10 rue du Dronckaert,59223,Roncq,813886603,Bienvenue chez 1000 AUTOS \n1000 AUTOS est spé...,Lundi au samedi de 09H00 à 12H00 et de 14H00 à...,Actif,24/05/2024,49,mon-sat,09:00 - 18:00
4,100% MOTO,1&3 rue pierre loti,42100,ST ETIENNE,452069610,DUCATI SAINT-ETIENNE\n\nDÉCOUVREZ NOS OCCASION...,du mardi au vendredi: 09h00-12h00 14h00-19h00 ...,Actif,02/11/2010,28,tue-fri,09:00 - 19:00


In [7]:
# save to csv
companies_df.to_csv('../../extracted_data/leboncoin/companies_normalized.csv')